In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
import math
import numpy as np
import re
import math
import time
from mongo import Mongo
repo = Mongo('trading_bot', 'company_debt')

In [33]:
class DebtScrapper:
    def __init__(self):
        pass
    def get_debt(self, url='https://mops.twse.com.tw/mops/web/t120sb02_q9', due=True):
        dfs = []
        driver = webdriver.Chrome('/Users/max/Downloads/')
        driver.get(url=url)

        #wait until the bond_id_1 appear
        wait = WebDriverWait(driver, 10)
        bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

        #input the first company id
        bond_id_1.send_keys('0000')

        #input the last company id
        bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
        bond_id_2.send_keys('9999')
        if due !=True:
            #undue debt button
            undue_debt_button_XPATH = '//*[@id="table01"]/center/form/table/tbody/tr[14]/td/input[2]'
            undue_debt_button = driver.find_element(By.XPATH, undue_debt_button_XPATH)
            undue_debt_button.click()
        
        else:
            pass
        #press the search button
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
        search_button.click()

        #__________________________________________________________________________________________________
        # the section to get table and see if there is more than 100 rows

        date_string = dt.datetime.now().strftime('%Y-%m-%d')
        how_many_month = self.calculate_month_difference(date_string)


        for i in range(how_many_month):
            tables_ID = 'table_0'
            try:
                tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
            except TimeoutException:
                no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                    print('no more data driver will close')
                    driver.quit()
                    break
                driver.quit()


            current_month_XPATH ='//*[@id="table01"]/center/form/table[1]/tbody/tr[1]/td/div/table/tbody/tr[2]/td'
            current_month = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH,current_month_XPATH )))
            debt_date = self.convert_roc_to_gregorian(current_month.text)

            raw_data = tables.text
            data = []
            for text in raw_data.split('\n'):
                splited_text = text.split(' ')
                data.append(splited_text)

            df= pd.DataFrame(data)
            stored_df = self.__process_df(raw_data, debt_date)
            dfs.append(stored_df)

            how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
            new_list = [i for i in how_many_page if i != '']
            page_str = new_list[0]
            final_page_str = int(page_str[1:len(page_str)-1])

            times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
            #__________________________________________________________________________________________________
            # the next page section
            if times_to_click_next_page_button >=1:
                for i in range(times_to_click_next_page_button):
                    next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
                    next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
                    try:
                        next_page_button.click()
                    except ElementNotInteractableException:
                        print('button is unclickable')
                        print('wait 10 sec and retrying')
                        time.sleep(10)
                    
                    tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
                    try:
                        tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
                    except TimeoutException:
                        print('unable to find the table')
                        print('wait 10 sec and retrying')
                        time.sleep(10)

                #__________________________________________________________________________________________________
                # the next month section
                try:    
                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                    if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                        print('no more data driver will close')
                        driver.quit()
                        break
                    else:
                        #jump to the else
                        continue
            #__________________________________________________________________________________________________
            # moving on to next month if there is no more than 100 rows
            else:
                try:    
                    raw_data = tables.text
                    df = self.__process_df(raw_data,debt_date)
                    dfs.append(df)

                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    print('Element not found, waiting and retrying')
                    time.sleep(10)
                    no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                    if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                        print('no more data driver will close')
                        driver.close()
                        # jump to the end of the code
                        break
                        
        
        driver.quit()
        final_df = pd.concat(dfs)
        final_df.drop_duplicates(inplace=True)
        return final_df 
    def calculate_month_difference(self,date_string):
        start_date = pd.to_datetime("2013-01-01")
        end_date = pd.to_datetime(date_string)

        num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 3

        return num_months




    def __process_df(self,raw_data,date):
        data = []
        for text in raw_data.split('\n'):
            splited_text = text.split(' ')
            data.append(splited_text)
        
        df = pd.DataFrame(data)
        df.dropna(axis=0,how='all', inplace=True)

        df.columns = df.iloc[0]

        df = df.iloc[1:]

        df = df.iloc[:,0:14]
        df.drop(columns=['券別','幣別'], inplace=True)

        df.columns = ['stock_id','債券種類',
    '公司名稱',
    '債券代碼',
    '債券簡稱',
    '發行日期',
    '票面利率',
    '到期日期',
    '債券期別',
    '幣別',
    '發行總額',
    '月底餘額',]
        df['Timestamp'] = date
        return df
    def get_all_debt(self):

        historical_undue_dfs = self.get_debt(url='https://mops.twse.com.tw/mops/web/t120sb02_q9')

        time.sleep(200)
        historical_due_dfs = self.get_debt(due=False, url='https://mops.twse.com.tw/mops/web/t120sb02_q9')

        present_undue_debt = self.get_debt(due=True, url='https://mops.twse.com.tw/mops/web/t120sb02_q10')

        present_due_debt = self.get_debt(due=False, url='https://mops.twse.com.tw/mops/web/t120sb02_q10')
        all_df = pd.concat([historical_due_dfs,historical_undue_dfs, present_due_debt,present_undue_debt ],axis=0 )
        all_df.drop_duplicates(inplace=True)
        self.historical_df = all_df
        return all_df
    

    def send_to_repo(self,dfs,repo):
        for _, row in dfs.iterrows():
            repo.send_document(row.to_dict())
    from datetime import datetime

    def convert_roc_to_gregorian(self,date_str):
        # Extract year and month
        date_str = date_str[6:]
        year_str, month_str = date_str.split(' 年 ')
        month_str = month_str.split(' 月份')[0]
        
        # Convert ROC year to Gregorian year
        year = int(year_str) + 1911
        
        # Create a datetime object
        timestamp = dt.datetime(year=int(year), month=int(month_str), day=1)
        
        return timestamp


        

In [34]:
s = DebtScrapper()
s.get_debt()


/var/folders/ty/0sfpr_454fxfc8pv3h3387m00000gq/T/ipykernel_15281/1582408543.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')


no more data driver will close


,stock_id,債券種類,公司名稱,債券代碼,債券簡稱,發行日期,票面利率,到期日期,債券期別,幣別,發行總額,月底餘額,Timestamp
1,1316,轉(交)換公司債,上曜,13164,上曜四,110/01/29,0.000000,115/01/29,4,新台幣,"400,000,000","236,200,000",2023-02-01
2,1316,轉(交)換公司債,上曜,13165,上曜五,110/02/01,0.000000,113/02/01,5,新台幣,"400,000,000","136,700,000",2023-02-01
3,1338,轉(交)換公司債,廣華-KY,13381,廣華一KY,110/01/19,0.000000,115/01/19,1,新台幣,"600,000,000","600,000,000",2023-02-01
4,1341,轉(交)換公司債,富林-KY,13411,富林一KY,110/05/13,0.000000,113/05/13,1,新台幣,"150,000,000","112,400,000",2023-02-01
5,1342,轉(交)換公司債,八貫,13421,八貫一,111/06/02,0.000000,114/06/02,1,新台幣,"200,000,000","72,900,000",2023-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,8042,轉(交)換公司債,金山電,80425,金山電五,107/07/10,0.000000,112/07/10,5,新台幣,"700,000,000","700,000,000",2018-06-01
3,8933,轉(交)換公司債,愛地雅,89335,愛地雅五,107/06/08,0.000000,112/06/08,5,新台幣,"400,000,000","400,000,000",2018-06-01
4,合計,,,,,,,,,,,,2018-06-01
1,8933,轉(交)換公司債,愛地雅,89335,愛地雅五,107/06/08,0.000000,112/06/08,5,新台幣,"400,000,000","400,000,000",2018-05-01


In [3]:
repo = Mongo(db='trading_bot',collection='company_debt')
repo.get_all_data()

,_id,債券種類,公司名稱,債券代碼,債券簡稱,發行日期,票面利率,到期日期,債券期別,幣別,發行總額,月底餘額
0,6476aea42deac4d94031f429,轉(交)換公司債,上曜,13164,上曜四,110/01/29,0.000000,115/01/29,4,新台幣,"400,000,000","236,200,000"
1,6476aea82deac4d94031f42a,轉(交)換公司債,上曜,13165,上曜五,110/02/01,0.000000,113/02/01,5,新台幣,"400,000,000","136,700,000"
2,6476aea82deac4d94031f42b,轉(交)換公司債,廣華-KY,13381,廣華一KY,110/01/19,0.000000,115/01/19,1,新台幣,"600,000,000","600,000,000"
3,6476aea92deac4d94031f42c,轉(交)換公司債,富林-KY,13411,富林一KY,110/05/13,0.000000,113/05/13,1,新台幣,"150,000,000","118,000,000"
4,6476aea92deac4d94031f42d,轉(交)換公司債,八貫,13421,八貫一,111/06/02,0.000000,114/06/02,1,新台幣,"200,000,000","100,400,000"
...,...,...,...,...,...,...,...,...,...,...,...,...
4132,6476b11e2deac4d94032044d,轉(交)換公司債,一詮精密,24864,一詮四,99/05/06,0.000000,104/05/06,四,,"2,200,000,000","2,174,700,000"
4133,6476b11e2deac4d94032044e,轉(交)換公司債,聚陽實業,14773,聚陽三,98/06/22,0.000000,103/06/22,3,,"600,000,000","30,000,000"
4134,6476b11e2deac4d94032044f,轉(交)換公司債,聚陽實業,14774,聚陽四,101/08/24,0.000000,106/08/24,4,,"700,000,000","688,600,000"
4135,6476b11f2deac4d940320450,轉(交)換公司債,億光電子,23934,億光四,98/12/04,0.000000,103/12/04,4,,,"2,500,000,000"


In [24]:
date = '查詢月份： 112 年 02 月份'
date = date[6:]
year = date[:3]


(year)

'112'

In [ ]:
# class DebtScrapper:
#     def __init__(self):
#         pass

#     def get_historical_due_debt(self):
#         dfs = []
#         driver = webdriver.Chrome('/Users/max/Downloads/')
#         driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q9')

#         #wait until the bond_id_1 appear
#         wait = WebDriverWait(driver, 10)
#         bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

#         #input the first company id
#         bond_id_1.send_keys('0000')

#         #input the last company id
#         bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
#         bond_id_2.send_keys('9999')

#         #press the search button
#         search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
#         search_button.click()

#         #__________________________________________________________________________________________________
#         # the section to get table and see if there is more than 100 rows
#         for i in range(100):
            
#             tables_ID = 'table_0'
#             try:
#                 tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
#             except TimeoutException:
#                 no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
#                 if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
#                     print('no more data driver will close')
#                     driver.quit()
#                     break
#             raw_data = tables.text
#             if not raw_data:  # This is the check for if there is table data.
#                 break  # If there is no table data, break the loop.
#             data = []
#             for text in raw_data.split('\n'):
#                 splited_text = text.split(' ')
#                 data.append(splited_text)

#             df= pd.DataFrame(data)
#             stored_df = self.__process_df(raw_data)
#             dfs.append(stored_df)

#             how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
#             new_list = [i for i in how_many_page if i != '']
#             page_str = new_list[0]
#             final_page_str = int(page_str[1:len(page_str)-1])

#             times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
#             #__________________________________________________________________________________________________
#             # the next page section
#             if times_to_click_next_page_button >=1:
#                 for i in range(times_to_click_next_page_button):
#                     next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
#                     next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
#                     try:
#                         next_page_button.click()
#                     except ElementNotInteractableException:
#                         print('button is unclickable')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)
                    
#                     tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
#                     try:
#                         tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
#                     except TimeoutException:
#                         print('unable to find the table')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)

#                 #__________________________________________________________________________________________________
#                 # the next month section
#                 try:    
#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('element not found retrying')
#                     time.sleep(10)
#             #__________________________________________________________________________________________________
#             # moving on to next month if there is no more than 100 rows
#             else:
#                 try:    
#                     raw_data = tables.text
#                     df = self.__process_df(raw_data)
#                     dfs.append(df)

#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('Element not found, waiting and retrying')
#                     time.sleep(10)


#         final_df = pd.concat(dfs)
#         final_df.drop_duplicates(inplace=True)
#         driver.quit()
#         return final_df 
#     def get_historical_undue_debt(self):
#         dfs = []
#         driver = webdriver.Chrome('/Users/max/Downloads/')
#         driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q9')

#         #wait until the bond_id_1 appear
#         wait = WebDriverWait(driver, 10)
#         bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

#         #input the first company id
#         bond_id_1.send_keys('0000')

#         #input the last company id
#         bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
#         bond_id_2.send_keys('9999')


#         #undue debt button
#         undue_debt_button_XPATH = '//*[@id="table01"]/center/form/table/tbody/tr[14]/td/input[2]'
#         undue_debt_button = driver.find_element(By.XPATH, undue_debt_button_XPATH)
#         undue_debt_button.click()
        

#         #press the search button
#         search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
#         search_button.click()

#         #__________________________________________________________________________________________________
#         # the section to get table and see if there is more than 100 rows

#         date_string = dt.datetime.now().strftime('%Y-%m-%d')
#         how_many_month = self.calculate_month_difference(date_string)


#         for i in range(how_many_month):
#             tables_ID = 'table_0'
#             try:
#                 tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
#             except TimeoutException:
#                 no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
#                 if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
#                     print('no more data driver will close')
#                     driver.quit()
#                     break
#                 driver.quit()
#             raw_data = tables.text
#             data = []
#             for text in raw_data.split('\n'):
#                 splited_text = text.split(' ')
#                 data.append(splited_text)

#             df= pd.DataFrame(data)
#             stored_df = self.__process_df(raw_data)
#             dfs.append(stored_df)

#             how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
#             new_list = [i for i in how_many_page if i != '']
#             page_str = new_list[0]
#             final_page_str = int(page_str[1:len(page_str)-1])

#             times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
#             #__________________________________________________________________________________________________
#             # the next page section
#             if times_to_click_next_page_button >=1:
#                 for i in range(times_to_click_next_page_button):
#                     next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
#                     next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
#                     try:
#                         next_page_button.click()
#                     except ElementNotInteractableException:
#                         print('button is unclickable')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)
                    
#                     tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
#                     try:
#                         tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
#                     except TimeoutException:
#                         print('unable to find the table')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)

#                 #__________________________________________________________________________________________________
#                 # the next month section
#                 try:    
#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
#                     if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
#                         print('no more data driver will close')
#                         driver.quit()
#                         break
#                     else:
#                         #jump to the else
#                         continue
#             #__________________________________________________________________________________________________
#             # moving on to next month if there is no more than 100 rows
#             else:
#                 try:    
#                     raw_data = tables.text
#                     df = self.__process_df(raw_data)
#                     dfs.append(df)

#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('Element not found, waiting and retrying')
#                     time.sleep(10)
#                     no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
#                     if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
#                         print('no more data driver will close')
#                         driver.close()
#                         # jump to the end of the code
#                         break
                        
        
#         driver.quit()
#         final_df = pd.concat(dfs)
#         final_df.drop_duplicates(inplace=True)
#         return final_df 
    
#     def get_present_due_debt(self):
#         dfs = []
#         driver = webdriver.Chrome('/Users/max/Downloads/')
#         driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q10')

#         #wait until the bond_id_1 appear
#         wait = WebDriverWait(driver, 10)
#         bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

#         #input the first company id
#         bond_id_1.send_keys('0000')

#         #input the last company id
#         bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
#         bond_id_2.send_keys('9999')

#         #press the search button
#         search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
#         search_button.click()

#         #__________________________________________________________________________________________________
#         # the section to get table and see if there is more than 100 rows
#         for i in range(100):
            
#             tables_ID = 'table_0'
#             try:
#                 tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
#             except TimeoutException:
#                 no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
#                 if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
#                     print('no more data driver will close')
#                     driver.quit()
#                     break
#             raw_data = tables.text
#             if not raw_data:  # This is the check for if there is table data.
#                 break  # If there is no table data, break the loop.
#             data = []
#             for text in raw_data.split('\n'):
#                 splited_text = text.split(' ')
#                 data.append(splited_text)

#             df= pd.DataFrame(data)
#             stored_df = self.__process_df(raw_data)
#             dfs.append(stored_df)

#             how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
#             new_list = [i for i in how_many_page if i != '']
#             page_str = new_list[0]
#             final_page_str = int(page_str[1:len(page_str)-1])

#             times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
#             #__________________________________________________________________________________________________
#             # the next page section
#             if times_to_click_next_page_button >=1:
#                 for i in range(times_to_click_next_page_button):
#                     next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
#                     next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
#                     try:
#                         next_page_button.click()
#                     except ElementNotInteractableException:
#                         print('button is unclickable')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)
                    
#                     tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
#                     try:
#                         tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
#                     except TimeoutException:
#                         print('unable to find the table')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)

#                 #__________________________________________________________________________________________________
#                 # the next month section
#                 try:    
#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('element not found retrying')
#                     time.sleep(10)
#             #__________________________________________________________________________________________________
#             # moving on to next month if there is no more than 100 rows
#             else:
#                 try:    
#                     raw_data = tables.text
#                     df = self.__process_df(raw_data)
#                     dfs.append(df)

#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('Element not found, waiting and retrying')
#                     time.sleep(10)


#         final_df = pd.concat(dfs)
#         final_df.drop_duplicates(inplace=True)
#         driver.quit()
#         return final_df 
#     def get_present_undue_debt(self):
#         dfs = []
#         driver = webdriver.Chrome('/Users/max/Downloads/')
#         driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q10')

#         #wait until the bond_id_1 appear
#         wait = WebDriverWait(driver, 10)
#         bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

#         #input the first company id
#         bond_id_1.send_keys('0000')

#         #input the last company id
#         bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
#         bond_id_2.send_keys('9999')

#         #undue debt button
#         undue_debt_button_XPATH = '//*[@id="table01"]/center/form/table/tbody/tr[14]/td/input[2]'
#         undue_debt_button = driver.find_element(By.XPATH, undue_debt_button_XPATH)
#         undue_debt_button.click()

#         #press the search button
#         search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
#         search_button.click()

#         #__________________________________________________________________________________________________
#         # the section to get table and see if there is more than 100 rows
#         for i in range(100):
            
#             tables_ID = 'table_0'
#             try:
#                 tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
#             except TimeoutException:
#                 no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
#                 if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
#                     print('no more data driver will close')
#                     driver.quit()
#                     break
#             raw_data = tables.text
#             if not raw_data:  # This is the check for if there is table data.
#                 break  # If there is no table data, break the loop.
#             data = []
#             for text in raw_data.split('\n'):
#                 splited_text = text.split(' ')
#                 data.append(splited_text)

#             df= pd.DataFrame(data)
#             stored_df = self.__process_df(raw_data)
#             dfs.append(stored_df)

#             how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
#             new_list = [i for i in how_many_page if i != '']
#             page_str = new_list[0]
#             final_page_str = int(page_str[1:len(page_str)-1])

#             times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
#             #__________________________________________________________________________________________________
#             # the next page section
#             if times_to_click_next_page_button >=1:
#                 for i in range(times_to_click_next_page_button):
#                     next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
#                     next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
#                     try:
#                         next_page_button.click()
#                     except ElementNotInteractableException:
#                         print('button is unclickable')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)
                    
#                     tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
#                     try:
#                         tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
#                     except TimeoutException:
#                         print('unable to find the table')
#                         print('wait 10 sec and retrying')
#                         time.sleep(10)

#                 #__________________________________________________________________________________________________
#                 # the next month section
#                 try:    
#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('element not found retrying')
#                     time.sleep(10)
#             #__________________________________________________________________________________________________
#             # moving on to next month if there is no more than 100 rows
#             else:
#                 try:    
#                     raw_data = tables.text
#                     df = self.__process_df(raw_data)
#                     dfs.append(df)

#                     #moving to the next month
#                     next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
#                     next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
#                     if next_month_button.is_displayed():
#                         next_month_button.click()
#                     WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
#                     time.sleep(2.5)
#                 except TimeoutException:
#                     print('Element not found, waiting and retrying')
#                     time.sleep(10)


#         final_df = pd.concat(dfs)
#         final_df.drop_duplicates(inplace=True)
#         driver.quit()
#         return final_df 
#     def calculate_month_difference(self,date_string):
#         start_date = pd.to_datetime("2013-01-01")
#         end_date = pd.to_datetime(date_string)

#         num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 3

#         return num_months




#     def __process_df(self,raw_data):
#         data = []
#         for text in raw_data.split('\n'):
#             splited_text = text.split(' ')
#             data.append(splited_text)
        
#         df = pd.DataFrame(data)
#         df.dropna(axis=0,how='all', inplace=True)

#         df.columns = df.iloc[0]

#         df = df.iloc[1:]
#         df.set_index('公司代號',inplace=True)
#         df.rename(index={'公司代號':'stock_id'})
#         df = df.iloc[:,0:13]
#         df.drop(columns=['券別','幣別'], inplace=True)

#         df.columns = ['債券種類',
#     '公司名稱',
#     '債券代碼',
#     '債券簡稱',
#     '發行日期',
#     '票面利率',
#     '到期日期',
#     '債券期別',
#     '幣別',
#     '發行總額',
#     '月底餘額',]

#         return df
#     def get_all_debt(self):

#         historical_due_dfs = self.get_historical_due_debt()

#         time.sleep(200)
#         historical_undue_dfs = self.get_historical_undue_debt()

#         present_due_debt = self.get_present_due_debt()

#         present_undue_debt = self.get_present_undue_debt
#         all_df = pd.concat([historical_due_dfs,historical_undue_dfs, present_due_debt,present_undue_debt ],axis=0 )
#         all_df.drop_duplicates(inplace=True)
#         self.historical_df = all_df
#         return all_df
    

#     def send_to_repo(self,dfs,repo):
#         for _, row in dfs.iterrows():
#             repo.send_document(row.to_dict())

        

In [59]:
s = DebtScrapper()
dfs = s.get_all_debt()
dfs

/var/folders/_c/ckm1ssmj50bbbdf8wksvbk_40000gr/T/ipykernel_95170/1995436242.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')


no more data driver will close


/var/folders/_c/ckm1ssmj50bbbdf8wksvbk_40000gr/T/ipykernel_95170/1995436242.py:120: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')
/var/folders/_c/ckm1ssmj50bbbdf8wksvbk_40000gr/T/ipykernel_95170/1995436242.py:256: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')


no more data driver will close


TypeError: cannot concatenate object of type '<class 'method'>'; only Series and DataFrame objs are valid

In [48]:
def send_to_repo(df, repo):
    for _, row in df.iterrows():
        repo.send_document(row.to_dict())

send_to_repo(dfs, repo)

In [ ]:
from mongo import Mongo
repo = Mongo(db='trading_bot', collection='company_debt')
repo.collection.drop()
for i in range(len(dfs)):
    row = dfs.iloc[i]
    dict_row = row.to_dict()

    repo.send_document(dict_row)
    

In [7]:
def process_df(raw_data):
    
    data = []
    for text in raw_data.split('\n'):
        splited_text = text.split(' ')
        data.append(splited_text)
    df = pd.DataFrame(data)

    df.columns = df.iloc[0]
    df.dropna(axis=0,how='all', inplace=True)
    df = df.iloc[1:]
    df.set_index('公司代號',inplace=True)
    df = df.iloc[:,0:13]
    df.drop(columns=['券別','幣別'], inplace=True)
    
    df.reset_index('公司代號', inplace=True)
    df.columns = ['stock_id','債券種類',
 '公司名稱',
 '債券代碼',
 '債券簡稱',
 '發行日期',
 '票面利率',
 '到期日期',
 '債券期別',
 '幣別',
 '發行總額',
 '月底餘額',]

    return df

In [55]:
from selenium.common.exceptions import TimeoutException
import re
import math
def present_debt_scrapper():
    dfs = []
    current_time = '112年01月'
    while True:
        
        try:
            driver = webdriver.Chrome('/Users/max/Downloads/')
            driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q9')


            #wait until the bond_id_1 appear
            wait = WebDriverWait(driver, 10)
            bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))
            
            #input the first company id
            bond_id_1.send_keys('0000')

            #input the last company id
            bond_id_2 = driver.find_element(By.NAME, 'bond_id_2')
            bond_id_2.send_keys('9999')


            #press the button
            search_button = driver.find_element(By.XPATH, '//*[@id="table01"]/center/form/input[7]')
            search_button.click()
            n=0
            clicks=0
            while True:
                try:
                    clicks+=1
                    if clicks > 100:
                        print('already make more than 100 request driver auto stop')
                        driver.close()
                        break
                    # n+=1
                    # if n == 10:
                    #     time.sleep(5)
                    #     n=0
                    tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="table_0"]/tbody')))
                    current_month = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="table01"]/center/form/table[1]/tbody/tr[1]/td/div/table/tbody/tr[2]/td')))
                    print(current_month.text)
                    raw_data = tables.text
                    
                    df = process_df(raw_data)
                    dfs.append(df)
                    how_many_page = (df.iloc[len(df)-1].dropna().iloc[1])
                    how_many_page = int(re.findall(r'\d+', how_many_page))

                    times_to_click_next_page_button =math.ceil(how_many_page / 100)-1

                    if times_to_click_next_page_button >=1:
                        



                    
                    
                    #moving to the next page
                    last_month_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]')))
                    if last_month_button.is_displayed():
                        last_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    print('Element not found, waiting and retrying')
                    time.sleep(10)
                    
                    if str(driver.find_element(By.XPATH, '//*[@id="table01"]/center/form/h4/font').text) == '查無符合條件之資料':
                        print('no more data driver will close')
                        driver.close()
                        break
                    
                

        except Exception as e:
            print(f'error occured : {e}')
            print('wait 10 second to run again')
            time.sleep(10)

        final_df = pd.concat(dfs,axis=0)
        final_df = final_df.drop_duplicates()
        return final_df

    #if encounter stall element not found = driver request it too fast the element are not shown
    #yet

/var/folders/_c/ckm1ssmj50bbbdf8wksvbk_40000gr/T/ipykernel_27731/3971727808.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')


查詢月份： 112 年 01 月份
查詢月份： 111 年 12 月份
查詢月份： 111 年 11 月份
查詢月份： 111 年 10 月份
查詢月份： 111 年 09 月份
查詢月份： 111 年 08 月份
查詢月份： 111 年 07 月份
查詢月份： 111 年 06 月份
查詢月份： 111 年 05 月份
查詢月份： 111 年 04 月份
查詢月份： 111 年 03 月份
查詢月份： 111 年 02 月份
查詢月份： 111 年 01 月份
查詢月份： 110 年 12 月份
查詢月份： 110 年 11 月份
查詢月份： 110 年 10 月份
查詢月份： 110 年 09 月份
查詢月份： 110 年 08 月份
查詢月份： 110 年 07 月份
查詢月份： 110 年 06 月份
查詢月份： 110 年 05 月份
查詢月份： 110 年 04 月份
查詢月份： 110 年 03 月份
查詢月份： 110 年 02 月份
查詢月份： 110 年 01 月份
查詢月份： 109 年 12 月份
查詢月份： 109 年 11 月份
查詢月份： 109 年 10 月份
查詢月份： 109 年 09 月份
查詢月份： 109 年 08 月份
查詢月份： 109 年 07 月份
查詢月份： 109 年 06 月份
查詢月份： 109 年 05 月份
查詢月份： 109 年 04 月份
查詢月份： 109 年 03 月份
查詢月份： 109 年 02 月份
查詢月份： 109 年 01 月份
查詢月份： 108 年 12 月份
查詢月份： 108 年 11 月份
查詢月份： 108 年 10 月份
查詢月份： 108 年 09 月份
查詢月份： 108 年 08 月份
查詢月份： 108 年 07 月份
查詢月份： 108 年 06 月份
查詢月份： 108 年 05 月份
查詢月份： 108 年 04 月份
查詢月份： 108 年 03 月份
查詢月份： 108 年 02 月份
查詢月份： 108 年 01 月份
查詢月份： 107 年 12 月份
查詢月份： 107 年 11 月份
查詢月份： 107 年 10 月份
查詢月份： 107 年 09 月份
查詢月份： 107 年 08 月份
查詢月份： 107 年 07 月份
查詢月份： 107 

KeyboardInterrupt: 

In [9]:
def get_undue_debt():
        dfs = []
        driver = webdriver.Chrome('/Users/max/Downloads/')
        driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q9')

        #wait until the bond_id_1 appear
        wait = WebDriverWait(driver, 10)
        bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

        #input the first company id
        bond_id_1.send_keys('0000')

        #input the last company id
        bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
        bond_id_2.send_keys('9999')

        #press the search button
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
        search_button.click()

        #__________________________________________________________________________________________________
        # the section to get table and see if there is more than 100 rows
        for i in range(100):
            
            tables_ID = 'table_0'
            try:
                tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
            except TimeoutException:
                no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                    print('no more data driver will close')
                    driver.close()
                    break
            raw_data = tables.text
            if not raw_data:  # This is the check for if there is table data.
                break  # If there is no table data, break the loop.
            data = []
            for text in raw_data.split('\n'):
                splited_text = text.split(' ')
                data.append(splited_text)

            df= pd.DataFrame(data)
            stored_df = process_df(raw_data)
            dfs.append(stored_df)

            how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
            new_list = [i for i in how_many_page if i != '']
            page_str = new_list[0]
            final_page_str = int(page_str[1:len(page_str)-1])

            times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
            #__________________________________________________________________________________________________
            # the next page section
            if times_to_click_next_page_button >=1:
                for i in range(times_to_click_next_page_button):
                    next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
                    next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
                    try:
                        next_page_button.click()
                    except ElementNotInteractableException:
                        print('button is unclickable')
                    
                    tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
                    try:
                        tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
                    except TimeoutException:
                        print('unable to find the table')

                #__________________________________________________________________________________________________
                # the next month section
                try:    
                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    print('element not found retrying')
                    time.sleep(10)
            #__________________________________________________________________________________________________
            # moving on to next month if there is no more than 100 rows
            else:
                try:    
                    raw_data = tables.text
                    df = process_df(raw_data)
                    dfs.append(df)

                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(1.5)
                except TimeoutException:
                    print('Element not found, waiting and retrying')
                    time.sleep(10)


        final_df = pd.concat(dfs)
        final_df.drop_duplicates(inplace=True)
        driver.close()
        return final_df 

In [10]:
get_undue_debt()

/var/folders/_c/ckm1ssmj50bbbdf8wksvbk_40000gr/T/ipykernel_51260/2329875448.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')


KeyboardInterrupt: 

In [ ]:
from mongo import Mongo
repo = Mongo(db='trading_bot', collection='company_debt')
for i in range(len(dfs)):
    df = dfs.iloc[i]
    dict = dfs.to_dict(df)
    repo.send_document(dict)
    

In [ ]:
final_df = pd.concat(dfs,axis=0)
final_df = final_df.drop_duplicates()
final_df.reset_index()

In [38]:
def calculate_month_difference(date_string):
        start_date = pd.to_datetime("2013-01-01")
        end_date = pd.to_datetime(date_string)

        num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) - 3

        return num_months

In [52]:
def get_present_due_debt():
        dfs = []
        driver = webdriver.Chrome('/Users/max/Downloads/')
        driver.get(url='https://mops.twse.com.tw/mops/web/t120sb02_q10')

        #wait until the bond_id_1 appear
        wait = WebDriverWait(driver, 10)
        bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

        #input the first company id
        bond_id_1.send_keys('0000')

        #input the last company id
        bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
        bond_id_2.send_keys('9999')

        #press the search button
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
        search_button.click()

        #__________________________________________________________________________________________________
        # the section to get table and see if there is more than 100 rows
        for i in range(100):
            
            tables_ID = 'table_0'
            try:
                tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
            except TimeoutException:
                no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                    print('no more data driver will close')
                    driver.quit()
                    break
            raw_data = tables.text
            if not raw_data:  # This is the check for if there is table data.
                break  # If there is no table data, break the loop.
            data = []
            for text in raw_data.split('\n'):
                splited_text = text.split(' ')
                data.append(splited_text)

            df= pd.DataFrame(data)
            stored_df = __process_df(raw_data)
            dfs.append(stored_df)

            how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
            new_list = [i for i in how_many_page if i != '']
            page_str = new_list[0]
            final_page_str = int(page_str[1:len(page_str)-1])

            times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
            #__________________________________________________________________________________________________
            # the next page section
            if times_to_click_next_page_button >=1:
                for i in range(times_to_click_next_page_button):
                    next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
                    next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
                    try:
                        next_page_button.click()
                    except ElementNotInteractableException:
                        print('button is unclickable')
                        print('wait 10 sec and retrying')
                        time.sleep(10)
                    
                    tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
                    try:
                        tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
                    except TimeoutException:
                        print('unable to find the table')
                        print('wait 10 sec and retrying')
                        time.sleep(10)

                #__________________________________________________________________________________________________
                # the next month section
                try:    
                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    print('element not found retrying')
                    time.sleep(10)
            #__________________________________________________________________________________________________
            # moving on to next month if there is no more than 100 rows
            else:
                try:    
                    raw_data = tables.text
                    df = self.__process_df(raw_data)
                    dfs.append(df)

                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    print('Element not found, waiting and retrying')
                    time.sleep(10)


        final_df = pd.concat(dfs)
        final_df.drop_duplicates(inplace=True)
        driver.quit()
        return final_df 

In [53]:
get_present_due_debt()

/var/folders/_c/ckm1ssmj50bbbdf8wksvbk_40000gr/T/ipykernel_95170/3076446039.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/max/Downloads/')


NameError: name 'self' is not defined

In [ ]:
def get_historical_undue_debt(self, url='https://mops.twse.com.tw/mops/web/t120sb02_q9', undue=True):
        dfs = []
        driver = webdriver.Chrome('/Users/max/Downloads/')
        driver.get(url=url)

        #wait until the bond_id_1 appear
        wait = WebDriverWait(driver, 10)
        bond_id_1 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_1')))

        #input the first company id
        bond_id_1.send_keys('0000')

        #input the last company id
        bond_id_2 = wait.until(EC.presence_of_element_located((By.NAME, 'bond_id_2')))
        bond_id_2.send_keys('9999')

        if undue:
            #undue debt button
            undue_debt_button_XPATH = '//*[@id="table01"]/center/form/table/tbody/tr[14]/td/input[2]'
            undue_debt_button = driver.find_element(By.XPATH, undue_debt_button_XPATH)
            undue_debt_button.click()
        
        else:
            pass
        #press the search button
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="table01"]/center/form/input[7]')))
        search_button.click()

        #__________________________________________________________________________________________________
        # the section to get table and see if there is more than 100 rows

        date_string = dt.datetime.now().strftime('%Y-%m-%d')
        how_many_month = self.calculate_month_difference(date_string)


        for i in range(how_many_month):
            tables_ID = 'table_0'
            try:
                tables = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID,tables_ID)))
            except TimeoutException:
                no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                    print('no more data driver will close')
                    driver.quit()
                    break
                driver.quit()
            raw_data = tables.text
            data = []
            for text in raw_data.split('\n'):
                splited_text = text.split(' ')
                data.append(splited_text)

            df= pd.DataFrame(data)
            stored_df = self.__process_df(raw_data)
            dfs.append(stored_df)

            how_many_page = df.iloc[len(df)-1].dropna().values.tolist()[1:]
            new_list = [i for i in how_many_page if i != '']
            page_str = new_list[0]
            final_page_str = int(page_str[1:len(page_str)-1])

            times_to_click_next_page_button =math.ceil(final_page_str / 100)-1
            #__________________________________________________________________________________________________
            # the next page section
            if times_to_click_next_page_button >=1:
                for i in range(times_to_click_next_page_button):
                    next_page_button_XPATH = '//*[@id="table01"]/center/form/div[1]/input[2]'
                    next_page_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,next_page_button_XPATH)))
                    
                    try:
                        next_page_button.click()
                    except ElementNotInteractableException:
                        print('button is unclickable')
                        print('wait 10 sec and retrying')
                        time.sleep(10)
                    
                    tables_XPATH = f'//*[@id="table_{i+1}"]/tbody'
                    
                    try:
                        tables = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,tables_XPATH )))
                    except TimeoutException:
                        print('unable to find the table')
                        print('wait 10 sec and retrying')
                        time.sleep(10)

                #__________________________________________________________________________________________________
                # the next month section
                try:    
                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                    if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                        print('no more data driver will close')
                        driver.quit()
                        break
                    else:
                        #jump to the else
                        continue
            #__________________________________________________________________________________________________
            # moving on to next month if there is no more than 100 rows
            else:
                try:    
                    raw_data = tables.text
                    df = self.__process_df(raw_data)
                    dfs.append(df)

                    #moving to the next month
                    next_month_XPATH = '//*[@id="table01"]/center/form/table[1]/tbody/tr[2]/td/div/input[1]'
                    next_month_button = wait.until(EC.presence_of_element_located((By.XPATH, next_month_XPATH)))
                    
                    if next_month_button.is_displayed():
                        next_month_button.click()
                    WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "dialog")))
                    time.sleep(2.5)
                except TimeoutException:
                    print('Element not found, waiting and retrying')
                    time.sleep(10)
                    no_more_data_XPATH = '//*[@id="table01"]/center/form/h4/font'
                    if str(driver.find_element(By.XPATH, no_more_data_XPATH).text) == '查無符合條件之資料':
                        print('no more data driver will close')
                        driver.close()
                        # jump to the end of the code
                        break
                        
        
        driver.quit()
        final_df = pd.concat(dfs)
        final_df.drop_duplicates(inplace=True)
        return final_df 